In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import json
from time import sleep
from PIL import Image
from glob import glob
from io import BytesIO
from easyocr import Reader
import urllib
import numpy as np
import os
from cv2 import imdecode, cvtColor, COLOR_RGB2GRAY


In [2]:
def open_chrome(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    return driver


In [3]:
def login(driver, USER_Name, PASSWORD):
    #login = driver.find_element(By.CLASS_NAME, "loginSignupTab")
    #login.click()
    
    user = driver.find_element(By.ID, "loginchecktype-username")

    sleep(3)
    user.send_keys(USER_Name)
    
    log_btn = driver.find_element(By.CLASS_NAME, "fbBtnColor")
    sleep(1)
    log_btn.click()
    
    password = driver.find_element(By.ID, "loginform-password")
    sleep(1)
    password.send_keys(PASSWORD)
    
    log_btn = driver.find_element(By.CLASS_NAME, "submitButton")
    log_btn.click()
    
    sleep(5)

In [4]:
def select_city(city='Amman'):
    driver.get('https://jo.opensooq.com/en/find?have_images=&allposts=&onlyPremiumAds=&onlyDonation=&onlyPrice=&onlyUrgent=&onlyShops=&onlyMemberships=&onlyBuynow=&memberId=&sort=record_posted_date.desc&term=&cat_id=&scid=59&city={}'.format(city))
    

In [5]:
def get_products(driver):
    soup = BeautifulSoup(driver.page_source)
    products_list = soup.find('ul', id="gridPostListing").find_all('li', class_='rectLi relative mb15')
    return products_list

In [6]:

def get_product_info(product):
    d={}

    try:
        name = product.find('h2').find('a').text.strip()
    except:
        return d
    
    try:
        category_list = [i.text.strip('\xa0| ') for i in product.find('div', class_='rectCatesLinks').find_all('span')]
    except:
        category_list = []
        
    href = product.find('h2', class_='fRight').find('a')['href']
    
    product_url = BASE_LINK + href
    product_page = BeautifulSoup(requests.get(product_url).text)
    
    try:
        address = [i.text.strip(' \n-') for i in product_page.find('div', class_='postSideForm').find_all('span')]
    except:
        address = ['address']
    


    d['Name'] = name
    

    d['address'] = '+'.join(address[1:])
    d['category'] = '+'.join(category_list)
    d['product url'] =  product_url
    try:
        img_url = product.find('span', class_='sellerPhone popupPhone inline vMiddle').find('img')['src']
        req = urllib.request.urlopen(img_url)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = imdecode(arr, -1)
        gray = cvtColor(img, COLOR_RGB2GRAY)
    
        phone_number = reader.readtext(gray)[0][1]
        d['phone number'] = phone_number
    except:
        d['phone number'] = np.nan
    
    return d

In [7]:
def display_phone_numbers(driver):
    for button in driver.find_elements(By.XPATH, '//span[@class="sellerPhone popupPhone inline vMiddle"]'):
        driver.execute_script("arguments[0].click();", button)
    


In [8]:
def next_page(driver):
    next_url = BeautifulSoup(driver.find_element(By.XPATH, '//li[@class="next"]').get_attribute('innerHTML')).find('a')['href']
    driver.get(BASE_LINK+next_url)

In [9]:
def get_main_category(df):
    
    with open('subcat2cat.json', 'r') as f:
        d = json.load(f)
    
    
    df['main category'] = df['category'].str.split('+').str[0].str.strip().map(d)
    
    return df

## start gather data

In [10]:

# change user name and password
USER_NAME = '962785551555'
PASSWORD = 'Aa@1234567'

# pick a city
CITY = 'Amman'

# name of file where data will be saved
SAVE_AT = 'Amman2'

# don't touch
url = 'https://jo.opensooq.com/en/login'
BASE_LINK = 'https://jo.opensooq.com'

if __name__ == '__main__':
    driver = open_chrome(url)
    login(driver, USER_NAME, PASSWORD)
    select_city(CITY)
    reader = Reader(['en'], gpu=False)
    
    with open('subcat2cat.json', 'r') as f:
        d = json.load(f)
        
    
    
    page = 1
    
    if not os.path.exists('./'+SAVE_AT):
        os.makedirs('./'+SAVE_AT)
    
    while True:
        display_phone_numbers(driver)
        sleep(2)
        
        product_list = get_products(driver)
       
        all_products = []
        for product in product_list:
            all_products.append(get_product_info(product))
            
        df = pd.DataFrame(all_products)
        
        df = get_main_category(df)
        df.to_csv('./'+SAVE_AT+'/'+CITY+'-{}.csv'.format(page), index=False)
        page += 1
        try:
            next_page(driver)
        except:
            print("end")
            break

Using CPU. Note: This module is much faster with a GPU.


KeyboardInterrupt: 

# Merge data together

In [13]:
# name of folder that you would like to gather all data stored in it
FOLDER_NAME = 'Gathered_data'



# A list of all joined files is returned
joined_list = glob("./"+FOLDER_NAME+'/*.csv')
  
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df = df.drop_duplicates()
df.to_csv('./Gathered_data/'+ FOLDER_NAME+'.csv', index=False)